In [1]:
import os, random, glob, pickle, collections, math, json
import numpy as np
import pandas as pd
from __future__ import division
from __future__ import print_function
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline 

from keras.models import Sequential, Model, load_model, model_from_json
from keras import layers
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense, LeakyReLU, Conv2D, Input, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K
K.set_image_dim_ordering('tf')

Using TensorFlow backend.


In [2]:
TRAIN_DIR = '../data/train/'
TEST_DIR = '../RFCN/JPEGImages/'
TRAIN_CROP_DIR = '../data/train_crop/'
TEST_CROP_DIR = '../data/test_stg1_crop/'
RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
CROP_MODEL = 'resnet50_FT_Hybrid_woNoF'
if os.getcwd().split('/')[-1] != CROP_MODEL:
    print('WRONG CROP_MODEL DIR!!!')
CHECKPOINT_DIR = './checkpoint/'
if not os.path.exists(CHECKPOINT_DIR):
    os.mkdir(CHECKPOINT_DIR)
LOG_DIR = './log/'
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
OUTPUT_DIR = './output/'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
CROP_CLASSES=FISH_CLASSES[:]
CROP_CLASSES.remove('NoF')
TESTFRAC = 0.9
CONF_THRESH = 0.8
ROWS = 224
COLS = 224
BATCHSIZE = 128
LEARNINGRATE = 1e-4
def featurewise_center(x):
    mean = np.mean(x, axis=0, keepdims=True)
    mean = np.mean(mean, axis=(1,2), keepdims=True)
    x_centered = x - mean
    return x_centered

def featurewise_mean(x):
    mean = np.mean(x, axis=0)
    mean = np.mean(mean, axis=(0,1))
    return mean

def preprocess_featurewise(x, featurewise_mean):
    x -= np.reshape(featurewise_mean, [1, 1, 3])
    return x

def load_img(path, bbox, target_size=None):
    img = Image.open(path)
    cropped = img.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
    width_cropped, height_cropped = cropped.size
    if height_cropped > width_cropped: cropped = cropped.transpose(method=2)  
    if target_size:
        cropped = cropped.resize((target_size[1], target_size[0]), Image.BILINEAR)
    return cropped

def get_best_model(checkpoint_dir = CHECKPOINT_DIR):
    files = glob.glob(checkpoint_dir+'*')
    val_losses = [float(f.split('-')[-1][:-5]) for f in files]
    index = val_losses.index(min(val_losses))
    print('Loading model from checkpoint file ' + files[index])
    model = load_model(files[index])
    model_name = files[index].split('/')[-1]
    print('Loading model Done!')
    return (model, model_name)

def data_from_df(df):
    X = np.ndarray((df.shape[0], ROWS, COLS, 3), dtype=np.uint8)
    y = np.zeros((df.shape[0], len(CROP_CLASSES)), dtype=K.floatx())
    i = 0
    for index,row in df.iterrows():
        image_file = row['image_file']
        fish = row['crop_class']
        bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
        cropped = load_img(TEST_DIR+image_file,bbox,target_size=(ROWS,COLS))
        X[i] = np.asarray(cropped)
        y[i,CROP_CLASSES.index(fish)] = 1
        i += 1
    return (X, y)

def data_load(name):
    file_name = 'data_'+name+'_{}_{}.pickle'.format(ROWS, COLS)
    if os.path.exists(OUTPUT_DIR+file_name):
        print ('Loading from file '+file_name)
        with open(OUTPUT_DIR+file_name, 'rb') as f:
            data = pickle.load(f)
        X = data['X']
        y = data['y']
    else:
        print ('Generating file '+file_name)
        
        if name=='train' or name=='valid': 
            df = GTbbox_df[GTbbox_df['split']==name]
        elif name=='all':
            df = GTbbox_df
        else:
            print('Invalid name '+name)
    
        X, y = data_from_df(df)

        data = {'X': X,'y': y}
        with open(OUTPUT_DIR+file_name, 'wb') as f:
            pickle.dump(data, f)
    X = X.astype(np.float32)
    X /= 255.
    return (X, y)

In [3]:
# GTbbox_df = ['image_file','image_class','crop_index','crop_class','xmin',''ymin','xmax','ymax','split']

file_name = 'GTbbox_df.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    GTbbox_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)       
    GTbbox_df = pd.DataFrame(columns=['image_file','image_class','crop_index','crop_class','xmin','ymin','xmax','ymax'])  

    crop_classes=FISH_CLASSES[:]
    crop_classes.remove('NoF')
    with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
        train_file_labels = f.readlines()
        
    for c in crop_classes:
        print(c)
        j = json.load(open('../data/BBannotations/{}.json'.format(c), 'r'))
        for l in j: 
            filename = l["filename"]
            head, image_file = os.path.split(filename)
            basename, file_extension = os.path.splitext(image_file)
            for i in range(len(train_file_labels)):
                if train_file_labels[i][:9] == basename:
                    image_class = train_file_labels[i][10:-1]
                    break
            image = Image.open(TEST_DIR+'/'+image_file)
            width_image, height_image = image.size
            for i in range(len(l["annotations"])):
                a = l["annotations"][i]
                xmin = (a["x"])
                ymin = (a["y"])
                width = (a["width"])
                height = (a["height"])
                xmax = xmin + width
                ymax = ymin + height
                assert max(xmin,0)<min(xmax,width_image)
                assert max(ymin,0)<min(ymax,height_image)
                GTbbox_df.loc[len(GTbbox_df)]=[image_file,image_class,i,a["class"],max(xmin,0),max(ymin,0),min(xmax,width_image),min(ymax,height_image)] 

    image_files_df = GTbbox_df[['image_file','image_class']]
    image_files_df.drop_duplicates(inplace = True)
    test_file_nb = image_files_df.shape[0] - int(math.ceil(image_files_df.shape[0]*TESTFRAC))
    train_files_df, valid_files_df = train_test_split(image_files_df, test_size=test_file_nb, random_state=1986, stratify=image_files_df.image_class)
    GTbbox_df['split'] = ['train' if image_file in train_files_df.image_file.tolist() else 'valid' for image_file in GTbbox_df.image_file]

    GTbbox_df.to_pickle(OUTPUT_DIR+file_name)
    print('Done')

# GTbbox_df.loc[(GTbbox_df['image_class']!='NoF') & (GTbbox_df['crop_class']!='NoF') & (GTbbox_df['image_class']!=GTbbox_df['crop_class'])]

Loading from file GTbbox_df.pickle


In [4]:
#Load data

X_all, y_all = data_load('all')
X_train, y_train = data_load('train')
X_valid, y_valid = data_load('valid')  
print('Loading data done.')

print('all sample ', X_all.shape[0])
print('train sample ', X_train.shape[0])
print('valid sample ', X_valid.shape[0])

print('featurewise mean of X_all is ', featurewise_mean(X_all))
print('featurewise mean of X_train is ', featurewise_mean(X_train))
print('featurewise mean of X_valid is ', featurewise_mean(X_valid))


Loading from file data_all_224_224.pickle
Loading from file data_train_224_224.pickle
Loading from file data_valid_224_224.pickle
Loading data done.
all sample  4371
train sample  3948
valid sample  423
featurewise mean of X_all is  [ 0.40696082  0.43766695  0.3947188 ]
featurewise mean of X_train is  [ 0.40635169  0.43670505  0.39435506]
featurewise mean of X_valid is  [ 0.41260275  0.4466438   0.39812338]


In [5]:
# #class weight = n_samples / (n_classes * np.bincount(y))
# class_weight_fish = dict(GTbbox_df.groupby('crop_class').size())
# class_weight = {}
# n_samples = GTbbox_df.shape[0]
# for key,value in class_weight_fish.items():
#         class_weight[CROP_CLASSES.index(key)] = n_samples / (len(CROP_CLASSES)*value)
# class_weight

train_df = GTbbox_df[GTbbox_df['split']=='train']
class_weight_fish = dict(train_df.groupby('crop_class').size())
class_weight = {}
ref = max(class_weight_fish.values())
for key,value in class_weight_fish.items():
    class_weight[CROP_CLASSES.index(key)] = ref/value
class_weight

{0: 1.0,
 1: 8.1035714285714278,
 2: 19.903508771929825,
 3: 24.138297872340427,
 4: 7.5132450331125824,
 5: 13.269005847953217,
 6: 3.1601671309192199}

In [6]:
#data preprocessing

all_mean = featurewise_mean(X_all)
def preprocessing_function(x):
    x -= np.reshape(all_mean, [1, 1, 3])
    return x

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocessing_function,
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCHSIZE, shuffle=True, seed=None)
# assert X_train.shape[0]%BATCHSIZE==0
steps_per_epoch = int(math.floor(X_train.shape[0]/BATCHSIZE))

X_valid_centered = X_valid - np.reshape(all_mean, [1, 1, 1, 3])


In [7]:
#callbacks

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=25, verbose=1, mode='auto')        

model_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR+'weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
        
learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)

tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=False, write_images=True)


In [ ]:
# Resnet50
#top layer training to end

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
predictions = Dense(len(CROP_CLASSES), kernel_initializer='glorot_normal', activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE:', BATCHSIZE)
# train the model on the new data for a few epochs
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=2000, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True)

In [12]:
### Resnet50
# fine tuning stg1
# 164 conv5c+top
# 142 conv5+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
# 6 conv2+conv3+conv4+conv5+top
# 2 conv1+conv2+conv3+conv4+conv5+top i.e. FROM SCRATCH
start_layer = 164

model, model_name = get_best_model()
# model = load_model(CHECKPOINT_DIR + 'weights.029-0.3772.hdf5')
# model_name = 'weights.029-0.3772.hdf5'
# print('model_name', model_name)

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE', BATCHSIZE)
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=2000, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=350)

Loading model from checkpoint file ./checkpoint/weights.349-0.8371.hdf5
Loading model Done!
BATCHSIZE 128
Epoch 351/2000
30/30 [==============================] - 58s - loss: 2.4181 - acc: 0.6773 - val_loss: 0.8089 - val_acc: 0.7305
Epoch 352/2000
30/30 [==============================] - 50s - loss: 2.5682 - acc: 0.6755 - val_loss: 0.8119 - val_acc: 0.7329
Epoch 353/2000
30/30 [==============================] - 51s - loss: 2.3679 - acc: 0.6687 - val_loss: 0.8040 - val_acc: 0.7281
Epoch 354/2000
30/30 [==============================] - 50s - loss: 2.4068 - acc: 0.6842 - val_loss: 0.8080 - val_acc: 0.7281
Epoch 355/2000
30/30 [==============================] - 50s - loss: 2.3399 - acc: 0.6745 - val_loss: 0.8062 - val_acc: 0.7305
Epoch 356/2000
30/30 [==============================] - 51s - loss: 2.1469 - acc: 0.6953 - val_loss: 0.7797 - val_acc: 0.7376
Epoch 357/2000
30/30 [==============================] - 52s - loss: 2.2256 - acc: 0.6874 - val_loss: 0.7862 - val_acc: 0.7352
Epoch 358/20

In [8]:
### Resnet50
# fine tuning stg2
# 164 conv5c+top
# 154 conv5b+conv5c+top
# 142 conv5+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
# 6 conv2+conv3+conv4+conv5+top
# 2 conv1+conv2+conv3+conv4+conv5+top i.e. FROM SCRATCH
start_layer = 154

# model, model_name = get_best_model()
model = load_model(CHECKPOINT_DIR + 'weights.539-0.3229.hdf5')
model_name = 'weights.539-0.3229.hdf5'
print('model_name', model_name)

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE', BATCHSIZE)
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=2000, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=540)

model_name weights.539-0.3229.hdf5
BATCHSIZE 128
Epoch 541/2000
30/30 [==============================] - 62s - loss: 0.4332 - acc: 0.9284 - val_loss: 0.3105 - val_acc: 0.8889
Epoch 542/2000
30/30 [==============================] - 54s - loss: 0.4026 - acc: 0.9302 - val_loss: 0.3051 - val_acc: 0.8960
Epoch 543/2000
30/30 [==============================] - 55s - loss: 0.4326 - acc: 0.9328 - val_loss: 0.3015 - val_acc: 0.8960
Epoch 544/2000
30/30 [==============================] - 55s - loss: 0.4204 - acc: 0.9328 - val_loss: 0.3014 - val_acc: 0.8983
Epoch 545/2000
30/30 [==============================] - 55s - loss: 0.3581 - acc: 0.9375 - val_loss: 0.2973 - val_acc: 0.8983
Epoch 546/2000
30/30 [==============================] - 54s - loss: 0.4443 - acc: 0.9318 - val_loss: 0.3022 - val_acc: 0.9007
Epoch 547/2000
30/30 [==============================] - 54s - loss: 0.3600 - acc: 0.9349 - val_loss: 0.2969 - val_acc: 0.9007
Epoch 548/2000
30/30 [==============================] - 55s - loss: 0

In [9]:
### Resnet50
# fine tuning stg3
# 164 conv5c+top
# 154 conv5b+conv5c+top
# 142 conv5+top i.e. conv5a+conv5b+conv5c+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
# 6 conv2+conv3+conv4+conv5+top
# 2 conv1+conv2+conv3+conv4+conv5+top i.e. FROM SCRATCH
start_layer = 142

model, model_name = get_best_model()
# model = load_model(CHECKPOINT_DIR + 'weights.539-0.3229.hdf5')
# model_name = 'weights.539-0.3229.hdf5'
# print('model_name', model_name)

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE', BATCHSIZE)
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=2000, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=631)

Loading model from checkpoint file ./checkpoint/weights.630-0.2407.hdf5
Loading model Done!
BATCHSIZE 128
Epoch 632/2000
30/30 [==============================] - 65s - loss: 0.1780 - acc: 0.9716 - val_loss: 0.2402 - val_acc: 0.9196
Epoch 633/2000
30/30 [==============================] - 55s - loss: 0.2034 - acc: 0.9701 - val_loss: 0.2449 - val_acc: 0.9196
Epoch 634/2000
30/30 [==============================] - 56s - loss: 0.1929 - acc: 0.9656 - val_loss: 0.2393 - val_acc: 0.9220
Epoch 635/2000
30/30 [==============================] - 55s - loss: 0.1905 - acc: 0.9675 - val_loss: 0.2357 - val_acc: 0.9243
Epoch 636/2000
30/30 [==============================] - 55s - loss: 0.1876 - acc: 0.9685 - val_loss: 0.2370 - val_acc: 0.9314
Epoch 637/2000
30/30 [==============================] - 56s - loss: 0.1896 - acc: 0.9665 - val_loss: 0.2255 - val_acc: 0.9338
Epoch 638/2000
30/30 [==============================] - 55s - loss: 0.1759 - acc: 0.9740 - val_loss: 0.2270 - val_acc: 0.9338
Epoch 639/20

In [12]:
### Resnet50
# fine tuning stg4
# 164 conv5c+top
# 154 conv5b+conv5c+top
# 142 conv5+top i.e. conv5a+conv5b+conv5c+top
# 132 conv4f+conv5+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
# 6 conv2+conv3+conv4+conv5+top
# 2 conv1+conv2+conv3+conv4+conv5+top i.e. FROM SCRATCH
start_layer = 132

model, model_name = get_best_model()
# model = load_model(CHECKPOINT_DIR + 'weights.539-0.3229.hdf5')
# model_name = 'weights.539-0.3229.hdf5'
# print('model_name', model_name)

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE', BATCHSIZE)
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=2000, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=684)

Loading model from checkpoint file ./checkpoint/weights.683-0.1587.hdf5
Loading model Done!
BATCHSIZE 128
Epoch 685/2000
30/30 [==============================] - 60s - loss: 0.0609 - acc: 0.9935 - val_loss: 0.1755 - val_acc: 0.9574
Epoch 686/2000
30/30 [==============================] - 57s - loss: 0.0651 - acc: 0.9901 - val_loss: 0.1737 - val_acc: 0.9622
Epoch 687/2000
30/30 [==============================] - 57s - loss: 0.0654 - acc: 0.9918 - val_loss: 0.1710 - val_acc: 0.9645
Epoch 688/2000
30/30 [==============================] - 57s - loss: 0.0706 - acc: 0.9908 - val_loss: 0.1687 - val_acc: 0.9622
Epoch 689/2000
30/30 [==============================] - 58s - loss: 0.0612 - acc: 0.9891 - val_loss: 0.1703 - val_acc: 0.9622
Epoch 690/2000
30/30 [==============================] - 58s - loss: 0.0598 - acc: 0.9909 - val_loss: 0.1718 - val_acc: 0.9598
Epoch 691/2000
30/30 [==============================] - 57s - loss: 0.0531 - acc: 0.9897 - val_loss: 0.1729 - val_acc: 0.9574
Epoch 692/20

In [ ]:
### Resnet50
# fine tuning stg5
# 164 conv5c+top
# 154 conv5b+conv5c+top
# 142 conv5+top i.e. conv5a+conv5b+conv5c+top
# 132 conv4f+conv5+top
# 122 conv4e+conv4f+conv5+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
# 6 conv2+conv3+conv4+conv5+top
# 2 conv1+conv2+conv3+conv4+conv5+top i.e. FROM SCRATCH
start_layer = 80

model, model_name = get_best_model()
# model = load_model(CHECKPOINT_DIR + 'weights.539-0.3229.hdf5')
# model_name = 'weights.539-0.3229.hdf5'
# print('model_name', model_name)

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE', BATCHSIZE)
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=2000, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=725)

Loading model from checkpoint file ./checkpoint/weights.724-0.1355.hdf5
Loading model Done!
BATCHSIZE 128
Epoch 726/2000
30/30 [==============================] - 78s - loss: 0.0306 - acc: 0.9964 - val_loss: 0.1675 - val_acc: 0.9669
Epoch 727/2000
30/30 [==============================] - 74s - loss: 0.0340 - acc: 0.9982 - val_loss: 0.1710 - val_acc: 0.9622
Epoch 728/2000
30/30 [==============================] - 75s - loss: 0.0325 - acc: 0.9953 - val_loss: 0.1621 - val_acc: 0.9645
Epoch 729/2000
30/30 [==============================] - 73s - loss: 0.0287 - acc: 0.9973 - val_loss: 0.1508 - val_acc: 0.9693
Epoch 730/2000
30/30 [==============================] - 75s - loss: 0.0216 - acc: 0.9964 - val_loss: 0.1520 - val_acc: 0.9669
Epoch 731/2000
30/30 [==============================] - 75s - loss: 0.0240 - acc: 0.9969 - val_loss: 0.1528 - val_acc: 0.9669
Epoch 732/2000
30/30 [==============================] - 73s - loss: 0.0426 - acc: 0.9971 - val_loss: 0.1484 - val_acc: 0.9716
Epoch 733/20

In [ ]:
### Resnet50
# fine tuning stg5
# 164 conv5c+top
# 154 conv5b+conv5c+top
# 142 conv5+top i.e. conv5a+conv5b+conv5c+top
# 132 conv4f+conv5+top
# 122 conv4e+conv4f+conv5+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
# 6 conv2+conv3+conv4+conv5+top
# 2 conv1+conv2+conv3+conv4+conv5+top i.e. FROM SCRATCH
start_layer = 38

model, model_name = get_best_model()
# model = load_model(CHECKPOINT_DIR + 'weights.539-0.3229.hdf5')
# model_name = 'weights.539-0.3229.hdf5'
# print('model_name', model_name)

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE', BATCHSIZE)
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=2000, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=760)

In [ ]:
#resume training

model, model_name = get_best_model()
# print('Loading model from weights.004-0.0565.hdf5')
# model = load_model(CHECKPOINT_DIR + 'weights.011-1.7062.hdf5')

# optimizer = Adam(lr=1e-4)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=2000, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered,y_valid),
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=350)

In [13]:
for i, layer in enumerate(model.layers):
    print(i,layer.name)

0 input_1
1 zero_padding2d_1
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 ac

In [13]:
#test prepare

test_model, test_model_name = get_best_model()
# print('Loading model from weights.004-0.0565.hdf5')
# test_model = load_model('./checkpoints/checkpoint2/weights.004-0.0565.hdf5')

def test_generator(df, mean, datagen = None, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            image_file = row['image_file']
            bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
            cropped = load_img(TEST_DIR+image_file,bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            x /= 255.
            if datagen is not None: x = datagen.random_transform(x)            
            x = preprocess_featurewise(x, mean)
            batch_x[i] = x
            i += 1
        if batch_index%50 == 0: print('batch_index', batch_index)
        yield(batch_x)
        
test_aug_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)


Loading model from checkpoint file ./checkpoint/weights.316-0.1940.hdf5
Loading model Done!


In [14]:
# train_mean = [0.37698776,  0.41491762,  0.38681713]
all_mean = featurewise_mean(X_all)
print('all_mean:', all_mean)

all_mean: [ 0.40696082  0.43766695  0.3947188 ]


In [16]:
#GTbbox_CROPpred_df = ['image_file','crop_index','crop_class','xmin','ymin','xmax','ymax','split'
#                      'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'logloss']

file_name = 'GTbbox_CROPpred_df_'+test_model_name+'_.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    GTbbox_CROPpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name) 
    nb_augmentation = 1
    if nb_augmentation ==1:
        test_preds = test_model.predict_generator(test_generator(df=GTbbox_df, mean=all_mean), 
                                                  steps=int(math.ceil(GTbbox_df.shape[0]/BATCHSIZE)), workers=1, pickle_safe=False)
    else:
        test_preds = np.zeros((GTbbox_df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
        for idx in range(nb_augmentation):
            print('{}th augmentation for testing ...'.format(idx+1))
            test_preds += test_model.predict_generator(test_generator(df=GTbbox_df, mean=all_mean, datagen=test_aug_datagen), 
                                                       steps=int(math.ceil(GTbbox_df.shape[0]/BATCHSIZE)), workers=1, pickle_safe=False)
        test_preds /= nb_augmentation

    CROPpred_df = pd.DataFrame(test_preds, columns=['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT'])
    GTbbox_CROPpred_df = pd.concat([GTbbox_df,CROPpred_df], axis=1)
    GTbbox_CROPpred_df['logloss'] = GTbbox_CROPpred_df.apply(lambda row: -math.log(row[row['crop_class']]), axis=1)
    GTbbox_CROPpred_df.to_pickle(OUTPUT_DIR+file_name) 

crop_valid_loss = GTbbox_CROPpred_df.loc[GTbbox_CROPpred_df['split']=='valid', 'logloss'].mean()
print('crop_valid_loss:', crop_valid_loss)
crop_all_loss = GTbbox_CROPpred_df['logloss'].mean()
print('crop_all_loss:', crop_all_loss)

Generating file GTbbox_CROPpred_df_weights.316-0.1940.hdf5_.pickle
batch_index 0
crop_valid_loss: 0.193993760436
crop_all_loss: 0.054481258386


In [18]:
print('all crop loss:', GTbbox_CROPpred_df['logloss'].mean())
print(GTbbox_CROPpred_df.groupby(['crop_class'])['logloss'].mean())

train_CROPpred_df = GTbbox_CROPpred_df[GTbbox_CROPpred_df['split']=='train']
print('train crop loss:', train_CROPpred_df['logloss'].mean())
print(train_CROPpred_df.groupby(['crop_class'])['logloss'].mean())

valid_CROPpred_df = GTbbox_CROPpred_df[GTbbox_CROPpred_df['split']=='valid']
print('valid crop loss:', valid_CROPpred_df['logloss'].mean())
print(valid_CROPpred_df.groupby(['crop_class'])['logloss'].mean())

all crop loss: 0.054481258386
crop_class
ALB      0.068493
BET      0.047069
DOL      0.000758
LAG      0.000098
OTHER    0.012689
SHARK    0.000090
YFT      0.059154
Name: logloss, dtype: float64
train crop loss: 0.0395334903092
crop_class
ALB      0.060490
BET      0.003700
DOL      0.000741
LAG      0.000056
OTHER    0.001819
SHARK    0.000082
YFT      0.023868
Name: logloss, dtype: float64
valid crop loss: 0.193993760436
crop_class
ALB      0.142912
BET      0.514118
DOL      0.000915
LAG      0.000449
OTHER    0.118579
SHARK    0.000159
YFT      0.371939
Name: logloss, dtype: float64


In [19]:
# RFCNbbox_RFCNpred_df = ['image_class','image_file','crop_index','xmin','ymin','xmax','ymax',
#                          'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                          'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']
# select fish_conf >= CONF_THRESH

file_name = 'RFCNbbox_RFCNpred_df_conf{:.2f}.pickle'.format(CONF_THRESH)
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    RFCNbbox_RFCNpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)        
    RFCNbbox_RFCNpred_df = pd.DataFrame(columns=['image_class','image_file','crop_index','xmin','ymin','xmax','ymax',
                                                  'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
                                                  'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']) 

    with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
        detections_full_AGNOSTICnms = pickle.load(f) 
    with open("../RFCN/ImageSets/Main/test.txt","r") as f:
        test_files = f.readlines()
    with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
        train_file_labels = f.readlines()
    assert len(detections_full_AGNOSTICnms) == len(test_files)
    
    count = np.zeros(len(detections_full_AGNOSTICnms))
    
    for im in range(len(detections_full_AGNOSTICnms)):
        if im%1000 == 0: print(im)
        basename = test_files[im][:9]
        if im<1000:
            image_class = '--'
        else:
            for i in range(len(train_file_labels)):
                if train_file_labels[i][:9] == basename:
                    image_class = train_file_labels[i][10:-1]
                    break
        image = Image.open(TEST_DIR+'/'+basename+'.jpg')
        width_image, height_image = image.size
        
        bboxes = []
        detects_im = detections_full_AGNOSTICnms[im]
        for i in range(len(detects_im)):
            if np.sum(detects_im[i,5:]) >= CONF_THRESH:
                bboxes.append(detects_im[i,:]) 
        count[im] = len(bboxes)
        if len(bboxes) == 0:
            ind = np.argmax(np.sum(detects_im[:,5:], axis=1))
            bboxes.append(detects_im[ind,:])
        bboxes = np.asarray(bboxes)

        for j in range(len(bboxes)):    
            bbox = bboxes[j]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            assert max(xmin,0)<min(xmax,width_image)
            assert max(ymin,0)<min(ymax,height_image)
            RFCNbbox_RFCNpred_df.loc[len(RFCNbbox_RFCNpred_df)]=[image_class,basename+'.jpg',j,max(xmin,0),max(ymin,0),
                                                                   min(xmax,width_image),min(ymax,height_image),
                                                                   bbox[4],bbox[5],bbox[6],bbox[7],bbox[8],bbox[9],bbox[10],bbox[11]]   
    
    RFCNbbox_RFCNpred_df.to_pickle(OUTPUT_DIR+file_name)      

Generating file RFCNbbox_RFCNpred_df_conf0.80.pickle
0
1000
2000
3000
4000


In [22]:
# RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df = ['image_class', 'image_file','crop_index','xmin','ymin','xmax','ymax',
#                                    'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                                    'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN',
#                                    'ALB_CROP', 'BET_CROP', 'DOL_CROP',
#                                    'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP',
#                                    'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

file_name = 'RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df_'+test_model_name+'_.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)  
    nb_augmentation = 1
    if nb_augmentation ==1:
        test_preds = test_model.predict_generator(test_generator(df=RFCNbbox_RFCNpred_df, mean=all_mean), 
                                                  steps=int(math.ceil(RFCNbbox_RFCNpred_df.shape[0]/BATCHSIZE)), workers=1, pickle_safe=False)
    else:
        test_preds = np.zeros((RFCNbbox_RFCNpred_df.shape[0], len(CROP_CLASSES)), dtype=K.floatx())
        for idx in range(nb_augmentation):
            print('{}th augmentation for testing ...'.format(idx+1))
            test_preds += test_model.predict_generator(test_generator(df=RFCNbbox_RFCNpred_df, mean=all_mean, datagen=test_aug_datagen), 
                                                       steps=int(math.ceil(RFCNbbox_RFCNpred_df.shape[0]/BATCHSIZE)), workers=1, pickle_safe=False)
        test_preds /= nb_augmentation

    CROPpred_df = pd.DataFrame(test_preds, columns=['ALB_CROP', 'BET_CROP', 'DOL_CROP', 'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP'])
    RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df = pd.concat([RFCNbbox_RFCNpred_df,CROPpred_df], axis=1)
    
    RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df['NoF'] = RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df['NoF_RFCN']
    for fish in ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']:
        RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df[fish] = RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df.apply(lambda row: (1-row['NoF_RFCN'])*row[[fish+'_CROP']], axis=1)

    RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df.to_pickle(OUTPUT_DIR+file_name) 

Generating file RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df_weights.316-0.1940.hdf5_.pickle
batch_index 0


In [23]:
# clsMaxAve and hybrid RFCNpred&CROPpred such that RFCNpred for NoF and CROPpred for fish
# test_pred_df = ['logloss','image_class','image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'split']
# RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df = ['image_class', 'image_file','crop_index','xmin','ymin','xmax','ymax',
#                                    'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                                    'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN',
#                                    'ALB_CROP', 'BET_CROP', 'DOL_CROP',
#                                    'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP',
#                                    'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

file_name = 'test_pred_df_Hybrid_'+test_model_name+'_.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    test_pred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)  
    with open("../RFCN/ImageSets/Main/test.txt","r") as f:
        test_files = f.readlines()
    
    test_pred_df = pd.DataFrame(columns=['logloss','image_class','image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT'])  
    for j in range(len(test_files)): 
        image_file = test_files[j][:-1]+'.jpg'
        test_pred_im_df = RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df.loc[RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df['image_file'] == image_file,
                                                                       ['image_class', 'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']]
        image_class = test_pred_im_df.iloc[0]['image_class']
        test_pred_im_df.drop('image_class', axis=1, inplace=True)
        max_score = test_pred_im_df.max(axis=1)
        max_cls = test_pred_im_df.idxmax(axis=1)
        test_pred_im_df['max_score'] = max_score
        test_pred_im_df['max_cls'] = max_cls
        test_pred_im_df['Count'] = test_pred_im_df.groupby(['max_cls'])['max_cls'].transform('count')
        idx = test_pred_im_df.groupby(['max_cls'])['max_score'].transform(max) == test_pred_im_df['max_score']
        test_pred_im_clsMax_df = test_pred_im_df.loc[idx,['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'Count']]
        test_pred_im_clsMax_array = test_pred_im_clsMax_df.values
        pred = np.average(test_pred_im_clsMax_array[:,:-1], axis=0, weights=test_pred_im_clsMax_array[:,-1], returned=False).tolist()
        if image_class!='--':
            ind = FISH_CLASSES.index(image_class)
            logloss = -math.log(pred[ind]) 
        else:
            logloss = np.nan
        test_pred_im_clsMaxAve = [logloss,image_class,image_file]
        test_pred_im_clsMaxAve.extend(pred)
        test_pred_df.loc[len(test_pred_df)]=test_pred_im_clsMaxAve
    
    image_split_df = GTbbox_df[['image_file','split']]
    image_split_df.drop_duplicates(inplace = True)
    test_pred_df = pd.merge(test_pred_df, image_split_df, how='outer', on='image_file')

    test_pred_df.to_pickle(OUTPUT_DIR+file_name) 

image_valid_loss = test_pred_df.loc[(test_pred_df['image_class']!='--') & (test_pred_df['split']=='valid'), 'logloss'].mean()
print('image_valid_loss:', image_valid_loss)
image_all_loss = test_pred_df.loc[test_pred_df['image_class']!='--', 'logloss'].mean()
print('image_all_loss:', image_all_loss)

Generating file test_pred_df_Hybrid_weights.316-0.1940.hdf5_.pickle
image_valid_loss: 0.176932356387
image_all_loss: 0.0717103426776


In [26]:
all_IMAGEpred_df = test_pred_df[test_pred_df['image_class']!='--']
print('all image loss:', all_IMAGEpred_df['logloss'].mean())
print(all_IMAGEpred_df.groupby(['image_class'])['logloss'].mean())

train_IMAGEpred_df = all_IMAGEpred_df[all_IMAGEpred_df['split']=='train']
print('train image loss:', train_IMAGEpred_df['logloss'].mean())
print(train_IMAGEpred_df.groupby(['image_class'])['logloss'].mean())

valid_IMAGEpred_df = all_IMAGEpred_df[all_IMAGEpred_df['split']=='valid']
print('valid image loss:', valid_IMAGEpred_df['logloss'].mean())
print(valid_IMAGEpred_df.groupby(['image_class'])['logloss'].mean())

all image loss: 0.0717103426776
image_class
ALB      0.079827
BET      0.058833
DOL      0.008619
LAG      0.000772
NoF      0.126442
OTHER    0.015035
SHARK    0.106109
YFT      0.052908
Name: logloss, dtype: float64
train image loss: 0.0449345139463
image_class
ALB      0.060760
BET      0.006116
DOL      0.009431
LAG      0.000753
OTHER    0.003747
SHARK    0.117218
YFT      0.027541
Name: logloss, dtype: float64
valid image loss: 0.176932356387
image_class
ALB      0.137899
BET      0.533292
DOL      0.001516
LAG      0.000930
OTHER    0.116252
SHARK    0.002207
YFT      0.282609
Name: logloss, dtype: float64


In [ ]:
#### visualization
# RFCNbbox_RFCNpred_CROPpred_df = ['image_class', 'image_file','crop_index','x_min','y_min','x_max','ymax',
#                                    'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                                    'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN'
#                                    'NoF_CROP', 'ALB_CROP', 'BET_CROP', 'DOL_CROP',
#                                    'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP']
#GTbbox_CROPpred_df = ['image_file','crop_index','crop_class','xmin','ymin','xmax','ymax',
#                      'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'logloss']
# test_pred_df = ['logloss','image_class','image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

for j in range(test_pred_df.shape[0]):
    image_logloss = test_pred_df.iat[j,0]
    image_class = test_pred_df.iat[j,1]
    image_file = test_pred_df.iat[j,2]
    if j<1000 and j%30== 0:
        pass
    else: 
        continue
    im = Image.open('../RFCN/JPEGImages/'+image_file)
    im = np.asarray(im)
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(im, aspect='equal')
    RFCN_dets = RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df.loc[RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df['image_file']==image_file]
    for index,row in RFCN_dets.iterrows():
        bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
        RFCN = [row['NoF_RFCN'],row['ALB_RFCN'],row['BET_RFCN'],row['DOL_RFCN'],row['LAG_RFCN'],row['OTHER_RFCN'],row['SHARK_RFCN'],row['YFT_RFCN']]
        CROP = [row['NoF'],row['ALB'],row['BET'],row['DOL'],row['LAG'],row['OTHER'],row['SHARK'],row['YFT']]
        score_RFCN = max(RFCN)
        score_CROP = max(CROP)
        index_RFCN = RFCN.index(score_RFCN)
        index_CROP = CROP.index(score_CROP)
        class_RFCN = FISH_CLASSES[index_RFCN]
        class_CROP = FISH_CLASSES[index_CROP]
        ax.add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1], fill=False, edgecolor='red', linewidth=2))
        ax.text(bbox[0], bbox[1] - 2, 'RFCN_{:s} {:.3f} \nHYBRID_{:s} {:.3f}'.format(class_RFCN, score_RFCN, class_CROP, score_CROP), bbox=dict(facecolor='red', alpha=0.5), fontsize=8, color='white')       
    GT_dets = GTbbox_CROPpred_df.loc[GTbbox_CROPpred_df['image_file']==image_file]
    for index,row in GT_dets.iterrows():
        bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
        CROP = [row['NoF'],row['ALB'],row['BET'],row['DOL'],row['LAG'],row['OTHER'],row['SHARK'],row['YFT']]
        score_CROP = max(CROP)
        index_CROP = CROP.index(score_CROP)
        class_CROP = FISH_CLASSES[index_CROP]
        ax.add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1], fill=False, edgecolor='green', linewidth=2))
        ax.text(bbox[0], bbox[3] + 40, 'GT_{:s} \nCROP_{:s} {:.3f}'.format(row['crop_class'], class_CROP, score_CROP), bbox=dict(facecolor='green', alpha=0.5), fontsize=8, color='white')
    ax.set_title(('Image {:s}    FISH {:s}    logloss {}').format(image_file, image_class, image_logloss), fontsize=10) 
    plt.axis('off')
    plt.tight_layout()
    plt.draw()

In [25]:
#temperature
T = 2.5
test_pred_array = test_pred_df[FISH_CLASSES].values
test_pred_T_array = np.exp(np.log(test_pred_array)/T)
test_pred_T_array = test_pred_T_array/np.sum(test_pred_T_array, axis=1, keepdims=True)
test_pred_T_df = pd.DataFrame(test_pred_T_array, columns=FISH_CLASSES)
test_pred_T_df = pd.concat([test_pred_df[['image_class','image_file']],test_pred_T_df], axis=1)

#test submission
submission = test_pred_T_df.loc[:999,['image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']]
submission.rename(columns={'image_file':'image'}, inplace=True)
sub_file = 'RFCN_AGONOSTICnms_'+RFCN_MODEL+'_'+CROP_MODEL+'_'+test_model_name+ \
'_clsMaxAve_conf{:.2f}_cropvalidloss{:.4f}_cropallloss{:.4f}_imagevalidloss{:.4f}_imageallloss{:.4f}_T{}.csv' \
.format(CONF_THRESH, crop_valid_loss, crop_all_loss, image_valid_loss, image_all_loss, T)
submission.to_csv(OUTPUT_DIR + sub_file, index=False)
print('Done!'+sub_file)

Done!RFCN_AGONOSTICnms_resnet101_rfcn_ohem_iter_30000_resnet55ss4_Hybrid_woNoF_weights.316-0.1940.hdf5_clsMaxAve_conf0.80_cropvalidloss0.1940_cropallloss0.0545_imagevalidloss0.1769_imageallloss0.0717_T2.5.csv
